In [1]:
from google.colab import files
uploaded = files.upload()  # Opens a file upload prompt


Saving PubChem_metabolite_SMILES_expProperties2_cleaned.csv to PubChem_metabolite_SMILES_expProperties2_cleaned.csv


In [2]:
!pip install --upgrade sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.0 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cud

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Load the dataset
data_pd = pd.read_csv('PubChem_metabolite_SMILES_expProperties2_cleaned.csv')

In [4]:
# Load tokenizer and model
checkpoint = "JuIm/SMILES_BERT"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint, output_attentions=True)  # Enable attention output

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/5.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at JuIm/SMILES_BERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Prepare the input sequences
sequences = data_pd['SMILES'].tolist()
solubility_logp = torch.tensor(data_pd['XLogP'].values, dtype=torch.float32)

In [6]:
# Tokenize SMILES sequences
tokenized_inputs = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
# Convert tokenized inputs to a dictionary of lists for splitting
tokenized_inputs_list = {key: val.tolist() for key, val in tokenized_inputs.items()}

In [9]:
# Split data into train and test sets
# Convert tokenized_inputs back to a dictionary of tensors before splitting
tokenized_inputs = {k: torch.tensor(v) for k, v in tokenized_inputs_list.items()}

# Get the input IDs (or other relevant features) from the tokenized inputs
input_ids = tokenized_inputs['input_ids']

# perform the split using input_ids and targets, and get the indices
train_indices, test_indices, _, _ = train_test_split(
    range(len(input_ids)), solubility_logp, test_size=0.3, random_state=42
)

train_inputs, test_inputs = input_ids[train_indices], input_ids[test_indices]
train_targets, test_targets = solubility_logp[train_indices], solubility_logp[test_indices]

In [12]:
# 🆕 Move inputs, attention masks, and targets to GPU
# Define device here
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_inputs = train_inputs.to(device)
test_inputs = test_inputs.to(device)
train_attention = tokenized_inputs['attention_mask'][train_indices].to(device)
test_attention = tokenized_inputs['attention_mask'][test_indices].to(device)
train_targets = train_targets.to(device)
test_targets = test_targets.to(device)

train_targets, test_targets = solubility_logp[train_indices], solubility_logp[test_indices]

In [13]:
# Custom dataset for PyTorch DataLoader
class SMILESDataset(Dataset):
    def __init__(self, input_ids, attention_mask, targets):
        self.input_ids = input_ids
        self.attention_mask = attention_mask # Add attention mask
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        # Return a dictionary with input_ids and attention_mask
        return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_mask[idx]}, self.targets[idx]

In [14]:
#train_dataset = SMILESDataset(train_inputs, tokenized_inputs['attention_mask'][train_indices], train_targets)
#test_dataset = SMILESDataset(test_inputs, tokenized_inputs['attention_mask'][test_indices], test_targets)
#train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [15]:
# 🆕 Use attention masks already moved to GPU
train_dataset = SMILESDataset(train_inputs, train_attention, train_targets)
test_dataset = SMILESDataset(test_inputs, test_attention, test_targets)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [16]:
# Define a regression head for fine-tuning
class BERTRegressor(nn.Module):
    def __init__(self, base_model):
        super(BERTRegressor, self).__init__()
        self.bert = base_model
        self.regressor = nn.Linear(base_model.config.hidden_size, 1)  # Regression layer

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # CLS token representation
        return self.regressor(pooled_output), outputs.attentions  # Return attentions

In [17]:
# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model = BERTRegressor(model).to(device)
#criterion = nn.MSELoss()
criterion = nn.MSELoss().to(device)  # 🆕 Move loss function to GPU
optimizer = optim.AdamW(fine_tuned_model.parameters(), lr=2e-5)

In [18]:
# Fine-tuning loop
num_epochs = 30
fine_tuned_model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    all_train_targets = []
    all_train_predictions = []

    for batch in train_loader:
        inputs, targets = batch
        inputs = {key: val.to(device) for key, val in inputs.items()}
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs, _ = fine_tuned_model(**inputs)

        # Check for NaN in outputs and targets before calculating loss
        if torch.isnan(outputs).any() or torch.isnan(targets).any():
            print("NaN detected in outputs or targets. Skipping this batch.")
            continue  # Skip this batch and move to the next

        loss = criterion(outputs.squeeze(), targets)
        loss.backward()

        # Clip gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(fine_tuned_model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()

        all_train_targets.extend(targets.cpu().numpy())
        all_train_predictions.extend(outputs.squeeze().cpu().detach().numpy())

    avg_loss = total_loss / len(train_loader)
    train_r2 = r2_score(all_train_targets, all_train_predictions)

    # Evaluate on test set (rest of the code remains the same)

    # Evaluate on test set
    fine_tuned_model.eval()
    all_test_targets = []
    all_test_predictions = []
    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = batch
            inputs = {key: val.to(device) for key, val in inputs.items()}
            targets = targets.to(device)

            outputs, _ = fine_tuned_model(**inputs)

            all_test_targets.extend(targets.cpu().numpy())
            all_test_predictions.extend(outputs.squeeze().cpu().numpy())

    test_r2 = r2_score(all_test_targets, all_test_predictions)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Train R^2: {train_r2:.4f}, Test R^2: {test_r2:.4f}")

print("Fine-tuning complete!")

RobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Epoch 1/30, Loss: 2.7172, Train R^2: 0.3723, Test R^2: 0.4993
Epoch 2/30, Loss: 1.8230, Train R^2: 0.5796, Test R^2: 0.6305
Epoch 3/30, Loss: 1.4570, Train R^2: 0.6737, Test R^2: 0.4898
Epoch 4/30, Loss: 1.1602, Train R^2: 0.7326, Test R^2: 0.6098
Epoch 5/30, Loss: 0.9798, Train R^2: 0.7734, Test R^2: 0.6849
Epoch 6/30, Loss: 0.7639, Train R^2: 0.8236, Test R^2: 0.6608
Epoch 7/30, Loss: 0.6131, Train R^2: 0.8585, Test R^2: 0.6838
Epoch 8/30, Loss: 0.4993, Train R^2: 0.8847, Test R^2: 0.6753
Epoch 9/30, Loss: 0.4510, Train R^2: 0.8959, Test R^2: 0.6901
Epoch 10/30, Loss: 0.4148, Train R^2: 0.9041, Test R^2: 0.7360
Epoch 11/30, Loss: 0.3689, Train R^2: 0.9150, Test R^2: 0.7156
Epoch 12/30, Loss: 0.3404, Train R^2: 0.9213, Test R^2: 0.7352
Epoch 13/30, Loss: 0.2862, Train R^2: 0.9338, Test R^2: 0.6831
Epoch 14/30, Loss: 0.3127, Train R^2: 0.9279, Test R^2: 0.7325
Epoch 15/30, Loss: 0.2562, Train R^2: 0.9409, Test R^2: 0.7330
Epoch 16/30, Loss: 0.2454, Train R^2: 0.9432, Test R^2: 0.7531
E